In [184]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [185]:
data = pd.read_csv('drive/MyDrive/all_v2.csv')

In [186]:
data.head()

,price,date,time,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,2018-02-19,20:00:21,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,2018-02-27,12:04:54,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,2018-02-28,15:44:00,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,2018-03-01,11:24:52,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,2018-03-01,17:42:43,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1


удаляю признак время, поскольку в констексте задачи, думаю он несущественный.

In [187]:
data = data.drop(['time'], axis=1)

In [188]:
data.date = pd.to_datetime(data.date)

По перцентилям определяю выбросы и удаляю их из выборки.  Есть значения по количеству комнат -2, их также удаляю. 

In [189]:
data = data[data.area>=17]

In [190]:
data = data[data.rooms != -2]

In [191]:
data = data.drop_duplicates()

In [192]:
data = data[data.area<=3000]
data = data[data.price>=200000]
data = data[data.price<=90000000]

In [181]:
#data.price.quantile(np.arange(.99, 1, .001))

In [178]:
#data.price.quantile(np.arange(0, 0.1, .001))

In [180]:
#data.area.quantile(np.arange(0, 0.03, .005))

In [179]:
#data.area.quantile(np.arange(.99, 1, .0001))

Преобразую временной признак.

In [193]:
data['day_delta'] = (data.date - data.date.min()).dt.days

In [194]:
data = data.drop('date', axis=1)

In [195]:
data.head(3)

,price,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type,day_delta
0,6050000,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1,0
1,8650000,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1,8
2,4000000,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1,9


Делю выборку на обучающую и тест с учетом временного ряда.

In [196]:
Y = data.price

In [197]:
X = data.drop('price', axis=1)

In [198]:
x_train = X[:int(len(Y)*0.7)]
x_test = X[int(len(Y)*0.7):]
y_train = Y[:int(len(Y)*0.7)]
y_test = Y[int(len(Y)*0.7):]

Нормализую данные.

In [202]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler() 
scaler.fit(x_train, y_train)
scaled_train = scaler.transform(x_train)
scaled_test = scaler.transform(x_test)

Строю baseline модель.

In [203]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(scaled_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [204]:
mean_absolute_error(y_test, model.predict(scaled_test))

2181911.32014653

Дополнительно ввожу метрику MAPE.

In [205]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [206]:
mape(y_test, model.predict(scaled_test))

50.04409315309821

Проверяю как изменится качество при использовании более сложной модели. 

In [207]:
from lightgbm import LGBMRegressor

In [208]:
lgb = LGBMRegressor()
lgb.fit(scaled_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [209]:
mean_absolute_error(y_test, lgb.predict(scaled_test))

1138732.3548951272

In [210]:
mape(y_test, lgb.predict(scaled_test))

22.924967018316696

In [211]:
y_test.describe()

count    1.561230e+06
mean     5.261904e+06
std      5.676226e+06
min      2.000000e+05
25%      2.450000e+06
50%      3.670000e+06
75%      6.000000e+06
max      9.000000e+07
Name: price, dtype: float64